## Bibliotecas

In [1]:
%pip install pandas
%pip install plotly_express
%pip install missingno
%pip install scipy
%pip install scikit-learn
%pip install prophet 
%pip install scikit-optimize 

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 0.0/100.3 kB ? eta -:--:--
     ----------- ------------------------- 30.7/100.3 kB 640.0 kB/s eta 0:00:01
     -------------------------------------- 100.3/100.3 kB 1.1 MB/s eta 0:00:00
  Obtaining dependency information for pyaml>=16.9 from https://files.pythonhosted.org/packages/56/db/01bc52d991716ccac7366b6e763fdaaf7b57910dbc2f85466c997f915284/pyaml-23.9.6-py3-none-any.whl.metadata
  Obtaining dependency information for PyYAML from https://files.pythonhosted.org/packages/b3/34/65bb4b2d7908044963ebf614fe0fdb080773fc7030d7e39c8d3eddcd4257/PyYAML-6.0.

## Importações

In [2]:
from math import sqrt
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.tree import DecisionTreeRegressor

## Carregamento de Dados

In [3]:
# Importa dos dados da Mobly
df = pd.read_csv("../documentos/outros/base_de_dados/mobly_data_official.csv", sep = ",")
df

,weekday_name,sku,unit_price,shipment_type,anchor_category,product_department,product_category,origin_country,process_costing,sku_color,price_status,winning_price,items_sold,avg_website_visits_last_week,stock_qty,month,day,year,value_dollar,rate_employability
0,6,549,1099.99,0,24,3,11,0,0,12,3,1488.07,0,17.285714,0,1,1,2020,4.0949,12.4
1,6,669,413.99,0,13,4,9,0,1,9,3,392.58,0,48.857143,56,1,1,2020,4.0949,12.4
2,6,745,949.99,0,10,4,12,0,1,17,3,1110.69,1,22.714286,0,1,1,2020,4.0949,12.4
3,6,129,2949.99,0,24,4,11,0,1,21,3,3457.04,0,2.800000,0,1,1,2020,4.0949,12.4
4,6,697,657.99,0,17,4,9,0,1,31,2,529.00,0,0.000000,0,1,1,2020,4.0949,12.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
740089,1,48,504.88,0,2,4,9,0,1,6,2,339.00,0,41.000000,61,7,3,2023,4.8222,8.0
740090,1,182,1717.95,0,23,4,11,0,1,12,4,1717.95,0,154.142857,1,7,3,2023,4.8222,8.0
740091,1,109,1659.88,0,5,4,9,0,1,22,2,1093.68,0,138.000000,16,7,3,2023,4.8222,8.0
740092,1,184,909.96,0,9,4,12,0,1,5,2,899.96,0,32.714286,20,7,3,2023,4.8222,8.0


#### Preparação dos dados


In [4]:
# Define a coluna que se quer prever
X = df.drop(['items_sold'], axis=1)
y = df['items_sold']

# Define os dados de treino e os de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Métrica de Avaliação dos Modelos Candidatos 

> O texto abaixo foi aproveitado do artefato 3 (3° sprint), conforme combinado com a Orientadora e Instrutor de programação.

Em princípio, as métricas que serão utilizadas para avaliar o desempenho dos modelos candidatos abaixo são explicadas no notebook _Modelagem_ e são: MAE (Erro Absoluto Médio), MSE (Erro Quadrático Médio), RMSE (Raiz Quadrada do Erro Médio) e _R² Score_ (Coeficiente de Determinação). Mas, mesmo observando todas essas métricas em cada um dos modelos, optamos por ter um foco maior na _R² Score_, uma vez que tal métrica possui uma fácil compreensão e interpretação. Além de que a métrica _R² Score_ consegue expressar uma medida da qualidade geral do modelo em questão, pois ela consegue explicar a variabilidade presente nos dados de saída, os quais são gerados pelo modelo. 

É válido destacar que a _R² Score_ tem uma extrema relevância para o desenvolvimento da mobil.ia, dado que o modelo precisa prever a quantidade de itens vendidos por SKU com a maior precisão possível (algo destacado tanto pelo TAPI quanto pelos parceiros da Mobly), de modo que a equipe de _Supply Chain_ da organização possa tomar as devidas decisões para equilibrar o abastecimento de estoque. A importância da precisão do modelo desenvolvido está centrada, principalmente, no quanto os resultados gerados pelo algoritmo pode afetar a empresa parceira, pois:
* Caso o algoritmo realize uma grande quantidade de previsões corretas (ou minimamente corretas), a Mobly poderá ter o seu lucro aumentado, pois terá produtos disponíveis para suprir as demandas dos seus clientes, sem desperdiçar materiais e, por consequência, sem perder o dinheiro investido. Além de que, por meio dessa disponibilidade, há a possibilidade da Mobly ter mais clientes e, consequentemente, aumentar o seu número de vendas e melhorar a imagem da marca;
* Caso o algoritmo realize uma grande quantidade de previsões incorretas, a Mobly poderá ter uma queda no seu lucro, pois a disponibilidade dos produtos pode variar, haja vista que o estoque não será bem abastecido. Além disso, a empresa pode lidar com o desperdício de materiais (e, como consequência, a perda de dinheiro), com a perda de clientes e com a desvalorização da marca.

Sendo assim, como a _R² Score_ consegue explicar a variabilidade dos dados de forma mais direta que as demais métricas, ela é de extrema importância para que o modelo preditivo desenvolvido consiga contribuir positivamente para a Mobly, É válido destacar que as outras métricas **não serão descartadas ou ignoradas**, apenas iremos analisar de forma mais concentrada o _R² Score_, além de que essa decisão foi validada com o parceiro de projeto na terceira sprint.

## Otimização dos hiperparâmetros dos modelos 

A otimização dos hiperparâmetros é o método responsável por ajustar as configurações próprias de cada modelo de predição, a fim de buscar aquelas que melhor se encaixam com o conjunto de dados utilizados para treinar esse algoritmo. Dessa forma, essa otimização é utilizada para melhorar o desempenho do modelo, promovendo uma melhor adaptação ao caso específico e ao seu conjunto de dados.

Dentre os algoritmos de otimização existentes, os mais utilizados são o **Random search** e o **Grid search**. O **Grid Search** é um algoritmo de otimização de hiperparâmetros relativamente simples e realiza a busca, em um conjunto de dados, de um ou mais hiperparâmetros, de modo a testar todas as combinações possíveis entre eles [1]. Os benefícios desse modelo consistem em uma exploração mais completa e robusta por meio de uma abordagem simples de implementar. Ao mesmo tempo, pode não ser tão viável em alguns casos, como, por exemplo, em _datasets_ muito grandes, podendo ser ineficiente e demorado. 

Já o **Random Search** possui um funcionamento muito parecido com o algoritmo anterior, porém, ao invés de testar todas as combinações, ele realiza uma busca aleatória, isto é, esse algoritmo testa combinações aleatórias de hiperparâmetros, de acordo com um número especificado de amostras [1]. Entre os benefícios desse algoritmo, é possível destacar a eficiência, uma vez que consegue ser aplicado em um tempo razoável (sendo ideal para datasets maiores) e, ao mesmo tempo, apresentar bons resultados. Entretanto, uma desvantagem desse algoritmo é que não há garantia de encontrar o melhor conjunto de hiperparâmetros, uma vez que não consegue abranger todos os tipos de combinações.

A partir disso, o grupo optou por aplicar os dois algoritmos de otimização citados para os modelos escolhidos. A aplicação do algoritmo de otimização será ajustada conforme o desempenho deles em cada modelo, e ao fim, será escolhido o melhor algoritmo para cada modelo e as respectivas métricas resultantes.

_[1] Miura, Lucas. Modelos de Predição | Otimização de Hiperparâmetros em Python. Medium, 12 abr. 2020. Disponível em: https://medium.com/turing-talks/modelos-de-predi%C3%A7%C3%A3o-otimiza%C3%A7%C3%A3o-de-hiperpar%C3%A2metros-em-python-3436fc55016e#. Acesso em 18 set. 2023._

> É válido ressaltar que o treinamento dos modelos de Regressão Linear, Árvore de Decisão e Random Forest foi realizado no caderno _Modelagem_ e, como combinado com o instrutor de programação e com a orientadora, serão aproveitados as informações documentadas desse notebook neste atual.

### Modelo Regressão Linear

Anteriormente, foi realizado o treinamento do modelo de Regressão Linear no notebook _Modelagem_ e, a partir desse treinamento, foi obtido os seguintes resultados:

* **Erro Absoluto Médio (MAE):** 2.3551262630973295
* **Erro Quadrático Médio (MSE):** 25.32130884490815
* **Raiz quadrada do erro-médio (RMSE):** 5.03202830326978
* **Coeficiente de Determinação (R²):** 0.30530596803262156

A partir dos resultados acima, optamos por aplicar a otimização por hiperparâmetros para observar se este modelo pode melhorar o seu desempenho. Sendo assim, abaixo será realizada essa otimização:

#### Parâmetros e declaração do modelo

In [5]:
# Evoca o modelo de regressão linear 
model_lr = LinearRegression()

# Define a grade de hiperparâmetros para pesquisa aleatória
param_lr = {
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'positive': [True, False],
}

#### Otimização por Random Search 

Inicialmente, aplicamos o **Random Search** com 50 iterações, a partir da configuração própria do algoritmo, o `n_iter`, para explorar um amplo espaço de hiperparâmetros.

In [6]:
# Cria um objeto RandomizedSearchCV para procurar os melhores parâmetros
random_search = RandomizedSearchCV(model_lr, param_lr, n_iter=50, cv=5, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1)

# Ajusta o modelo aos dados
random_search.fit(X_train, y_train)

c:\Users\Inteli\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 8 is smaller than n_iter=50. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=5, estimator=LinearRegression(), n_iter=50, n_jobs=-1,
                   param_distributions={'copy_X': [True, False],
                                        'fit_intercept': [True, False],
                                        'positive': [True, False]},
                   random_state=42, scoring='neg_mean_squared_error')

##### Desempenho e melhores parâmetros do Random Search

In [7]:
# Usa o modelo com os melhores parâmetros para fazer previsões nos dados de teste
y_pred = random_search.predict(X_test)

# Calcula as métricas com base nos dados de teste
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)

# Imprime o desempenho do modelo com os melhores parâmetros
print("Melhores hiperparâmetros:", random_search.best_params_)
print(f"Erro Absoluto Médio (MAE): {mae}")
print(f"Erro Quadrático Médio (MSE): {mse}")
print(f"Raiz do Erro Quadrático Médio (RMSE): {rmse}")
print(f"R2 Score: {r2}")

Melhores hiperparâmetros: {'positive': False, 'fit_intercept': True, 'copy_X': True}
Erro Absoluto Médio (MAE): 2.2638277970533385
Erro Quadrático Médio (MSE): 20.561296084045583
Raiz do Erro Quadrático Médio (RMSE): 4.534456536790884
R2 Score: 0.32930899142005177


#### Otimização por Grid Search 


In [8]:
# Cria um objeto GridSearchCV para procurar os melhores parâmetros
grid_search = GridSearchCV(model_lr, param_lr, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Ajusta o modelo aos dados de treinamento
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LinearRegression(), n_jobs=-1,
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'positive': [True, False]},
             scoring='neg_mean_squared_error')

##### Desempenho e melhores parâmetros do Grid Search

In [9]:
# Usa o modelo com os melhores parâmetros para fazer previsões nos dados de teste
y_pred = grid_search.predict(X_test)

# Calcula as métricas com base nos dados de teste
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)

# Imprime os melhores parâmetros encontrados
print("Melhores hiperparâmetros:", grid_search.best_params_)
print(f"Erro Absoluto Médio (MAE): {mae}")
print(f"Erro Quadrático Médio (MSE): {mse}")
print(f"Raiz do Erro Quadrático Médio (RMSE): {rmse}")
print(f"R2 Score: {r2}")

Melhores hiperparâmetros: {'copy_X': True, 'fit_intercept': True, 'positive': False}
Erro Absoluto Médio (MAE): 2.2638277970533385
Erro Quadrático Médio (MSE): 20.561296084045583
Raiz do Erro Quadrático Médio (RMSE): 4.534456536790884
R2 Score: 0.32930899142005177


#### Conclusão

Utilizamos os seguintes hiperparâmetro para o **Random Search** e o **Grid Search**:

  * `positive`: Quando definido como `True`, impõe a restrição de que os coeficientes do modelo devem ser não negativos. Isso é útil quando você tem conhecimento prévio de que os coeficientes devem ser positivos [2].

  * `fit_intercept`: Este parâmetro determina se a intercepção (ou viés) deve ser ajustada no modelo linear. Se for definido como `True`, o modelo ajustará a intercepção. Se for definido como `False`, o modelo não terá uma intercepção e passará pela origem [2]. 

  * `copy_X`: Este parâmetro determina se os dados de entrada (X) devem ser copiados antes do ajuste do modelo. Quando definido como `True`, uma cópia dos dados é feita para evitar modificações indesejadas. Quando definido como `False`, o modelo ajusta os dados diretamente [2]. 

Após utilizarmos o algoritmo de otimização de hiperparâmetro **Random Search**, observamos que os melhores hiperparâmetros encontrados foram:

`{'positive': False, 'fit_intercept': True, 'copy_X': True}`

Além disso, as métricas de desempenho do modelo de Regressão Linear foram as seguintes:

* **Erro Absoluto Médio (MAE):** 2.2638277970533385
* **Erro Quadrático Médio (MSE):** 20.561296084045583
* **Raiz do Erro Quadrático Médio (RMSE):** 4.534456536790884
* **R2 Score:** 0.32930899142005177


Em seguida, foi aplicado o **Grid Search** para uma busca mais exaustiva de hiperparâmetros. Os melhores hiperparâmetros encontrados foram:

`{'copy_X': True, 'fit_intercept': True, 'positive': False}`

Surpreendentemente, os resultados após a otimização com o **Grid Search** foram idênticos aos obtidos com o **Random Search**:

* **Erro Absoluto Médio (MAE):** 2.2638277970533385
* **Erro Quadrático Médio (MSE):** 20.561296084045583
* **Raiz do Erro Quadrático Médio (RMSE):** 4.534456536790884
* **R2 Score:** 0.32930899142005177

É evidente que, mesmo após a otimização de hiperparâmetros, as métricas de desempenho do modelo de Regressão Linear não apresentaram uma melhoria significativa. Isso ressalta a conclusão de que, para o caso deste projeto em específico, a Regressão Linear pode não ser o modelo mais apropriado, haja vista que o modelo candidato definido no notebook _Modelagem_ apresentou um _R²_, o qual corresponde a principal métrica que estamos utilizando para avaliar os modelos, de, aproximadamente, 68%, bem mais alto do que o valor dessa mesma métrica gerado pela otimização do modelo de Regressão Linear, que é, aproximadamente, 32%. Portanto, considerando esses resultados, pode ser apropriado explorar modelos mais complexos ou adequados para o problema em questão.

[2] scikit-learn. Scikit-learn: Machine Learning in Python. Versão 0.24.2. 2021. Disponível em: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

### Modelo Árvore de Decisão


Anteriormente, foi realizado o treinamento do modelo de Árvore de Decisão no notebook _Modelagem_ e, a partir desse treinamento, foi obtido os seguintes resultados:

* **Erro Absoluto Médio (MAE):** 2.3976261886782666
* **Erro Quadrático Médio (MSE):** 26.63081900872144
* **Raiz quadrada do erro-médio (RMSE):** 5.160505693119759
* **Coeficiente de Determinação (R²):** 0.269379353765814

A partir dos resultados acima, optamos por aplicar a otimização por hiperparâmetros para observar se este modelo pode melhorar o seu desempenho. Sendo assim, abaixo será realizada essa otimização:

#### Parâmetros e declaração do modelo

In [10]:
# Cria o modelo de árvore de decisão de regressão
model_dt = DecisionTreeRegressor()

# Define as distribuições para amostrar hiperparâmetros
param_dt = {
    'max_depth': np.arange(3, 15),
    'min_samples_split': np.arange(2, 21),
    'min_samples_leaf': np.arange(1, 11)
}

#### Otimização por Random Search 


Assim como ocorreu no modelo de Regressão Linear, aplicamos o algoritmo de otimização de hiperparâmetro **Random Search** no modelo de Árvore de Decisão com 50 iterações. Essa escolha de iterações está atrelada a possibilidade de se observar um amplo espaço de hiperparâmetros.

In [11]:
# Usa RandomizedSearchCV para encontrar hiperparâmetros aleatórios
random_search = RandomizedSearchCV(model_dt, param_dt, n_iter=50, cv=5, scoring='neg_mean_squared_error', verbose=1, random_state=42)
random_search.fit(X_train, y_train)

# Obtém os melhores hiperparâmetros encontrados
best_hyperparameters = random_search.best_params_

Fitting 5 folds for each of 50 candidates, totalling 250 fits


##### Desempenho e melhores parâmetros do Random Search

In [12]:
# Treina o modelo com os melhores hiperparâmetros encontrados
y_pred = DecisionTreeRegressor(**best_hyperparameters)
y_pred.fit(X_train, y_train)

# Faz previsões no conjunto de teste
predictions = y_pred.predict(X_test)

# Avalia o desempenho do modelo com os melhores hiperparâmetros
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)  
mae = mean_absolute_error(y_test, predictions)  
r2 = r2_score(y_test, predictions) 

print("Melhores hiperparâmetros:", random_search.best_params_)
print(f"Erro Absoluto Médio (MAE): {mae}")
print(f"Erro Quadrático Médio (MSE): {mse}")
print(f"Raiz do Erro Quadrático Médio (RMSE): {rmse}")
print(f"R2 Score: {r2}")

Melhores hiperparâmetros: {'min_samples_split': 3, 'min_samples_leaf': 9, 'max_depth': 14}
Erro Absoluto Médio (MAE): 1.635208667994986
Erro Quadrático Médio (MSE): 10.968605076594836
Raiz do Erro Quadrático Médio (RMSE): 3.3118884456748896
R2 Score: 0.6422139552163348


#### Otimização por Grid Search


In [13]:
# Usa GridSearchCV para encontrar os melhores hiperparâmetros
grid_search = GridSearchCV(model_dt, param_dt, cv=5, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)

# Obtém os melhores hiperparâmetros encontrados
best_hyperparameters = grid_search.best_params_

Fitting 5 folds for each of 2280 candidates, totalling 11400 fits


##### Desempenho e melhores parâmetros do Grid Search

In [ ]:
# Treina o modelo com os melhores hiperparâmetros encontrados
y_pred = DecisionTreeRegressor(**best_hyperparameters)
y_pred.fit(X_train, y_train)

# Faz previsões no conjunto de teste
predictions = y_pred.predict(X_test)

# Avalia o desempenho do modelo com os melhores hiperparâmetros
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)  
mae = mean_absolute_error(y_test, predictions)  
r2 = r2_score(y_test, predictions) 

print("Melhores hiperparâmetros:", grid_search.best_params_)
print(f"Erro Absoluto Médio (MAE): {mae}")
print(f"Erro Quadrático Médio (MSE): {mse}")
print(f"Raiz do Erro Quadrático Médio (RMSE): {rmse}")
print(f"R2 Score: {r2}")

Melhores hiperparâmetros: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 2}
Erro Absoluto Médio (MAE): 1.7889991501447182
Erro Quadrático Médio (MSE): 12.851929378070077
Raiz do Erro Quadrático Médio (RMSE): 3.5849587693682166
R2 Score: 0.580781608243827


#### Conclusão

Após utilizarmos o algoritmo de otimização por hiperparâmetros **Random Search**, observamos que os melhores hiperparâmetros encontrados foram:

`{'min_samples_split': 3, 'min_samples_leaf': 9, 'max_depth': 14}`

A partir da obtenção dos hiperparâmetros acima, através do **Random Search** aplicado ao algoritmo de Árvore de Decisão, é necessário compreender, de maneira abrangente, o significado de cada um deles, bem como a capacidade que possuem de afetar o desempenho do modelo de Árvore de Decisão. Nesse sentido, abaixo será definido cada um dos hiperparâmetros:

* **min_samples_split:** Trata-se de um hiperparâmetro  que define o "número mínimo de amostras necessárias para dividir um nó interno" [3]. Esse hiperparâmetro controla o quão "criteriosa" a árvore deve ser ao decidir dividir um nó interno em subnós. No caso do modelo em questão, o número mínimo de amostras necessárias para dividir um nó interno é três.
* **min_samples_leaf:** Esse hiperparâmetro trata-se do "número mínimo de amostras necessárias para estar em um nó folha" [3]. Esse hiperparâmetro afeta o tamanho mínimo permitido para um nó folha. Sendo assim, neste modelo o número mínimo de amostras necessárias para estar em um nó fola é nove.
* **max_depth:** Por fim, esse hiperparâmetro define a "profundidade máxima das árvores" [3]. A profundidade de uma árvore de decisão determina até que ponto ela pode se ramificar, o que impacta a complexidade do modelo. A partir disso, a profundidade máxima deste modelo é 14.

Além disso, as métricas de desempenho do modelo de Árvore de Decisão, com o **Random Search** aplicado, foram as seguintes:

* **Erro Absoluto Médio (MAE):** 1.6351466432565025
* **Erro Quadrático Médio (MSE):** 10.969213266190533
* **Raiz quadrada do erro-médio (RMSE):** 3.311980263556915
* **Coeficiente de Determinação (R²):** 0.6421941166180443 

Através desses resultados é possível notar que a otimização de hiperparâmetros do algoritmo **Random Search** é bastante eficiente, pois o valor da métrica principal, o _R²_, é de 26% aproximadamente no modelo sem otimização e com a otimização aplicada ao modelo, o valor de _R²_ aumenta para, aproximadamente, 64%. Esse resultado, quando comparado com a otimização máxima do modelo de Regressão Linear, indica que o modelo de Árvore de Decisão possui um melhor desempenho com os dados utilizados neste projeto. Porém, quando este algoritmo é comparado com o modelo **Random Forest** sem nenhuma otimização, o valor de _R²_ é menor, uma vez que tal métrica do **Random Forest** é de 68% aproximadamente. Com isso, é possível encontrar outra forma de otimizar ainda mais o _R²_ do modelo de Árvore de Decisão. 

Em seguida, foi aplicado o **Grid Search** para uma busca mais exaustiva de hiperparâmetros. Os melhores hiperparâmetros encontrados foram:

`{'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 5}`

Anteriormente foi explicado a definição desses mesmos hiperparâmetros, porém é válido ressaltar o que cada resultado significa. Com isso, o `max_depth` indica que a profundidade máxima das árvores é dez. Já o `min_samples_leaf` define que o número mínimo de amostras necessárias para estar em um nó folha é quatro. Por fim, o `min_samples_split` indica que o número mínimo de amostras necessárias para dividir um nó interno é cinco. 

Com isso, as métricas obtidas através do **Grid Search**, aplicado ao modelo de Árvore de Decisão, foram:
* **Erro Absoluto Médio (MAE):** 1.7888623000764408
* **Erro Quadrático Médio (MSE):** 12.850584895123045
* **Raiz quadrada do erro-médio (RMSE):** 3.584771247251775
* **Coeficiente de Determinação (R²):** 0.5808254640698448

Os resultados das métricas apresentados acima evidenciam que o **Grid Search** oferece uma melhoria para o modelo de Árvore de Decisão, haja vista que o _R²_ gerado é maior (58%) que a mesma métrica do modelo sem otimização (26%). Mas, em relação à otimização anterior, é evidente que o _R²_ é maior no algoritmo **Random Search**, que é 64%, do que neste algoritmo, que é 58%. 

Portanto, é válido ressaltar que mesmo com as duas otimizações aplicadas, o modelo de Árvore de Decisão não se comportou melhor que o modelo de **Random Forest**.

_[3] Faboci, Bruno. Otimizando Random Forest com GridSearch. LinkedIn. Disponível em: https://www.linkedin.com/pulse/otimizando-random-forest-com-gridsearch-bruno-faboci/?originalSubdomain=pt. Acesso em: 22 de setembro de 2023._

### Modelo Random Forest

Anteriormente, foi realizado o treinamento do modelo Random Forest no notebook _Modelagem_ e, a partir desse treinamento, decidimos que tal algoritmo seria o nosso modelo candidato, haja vista que ele gerou a métrica principal de avaliação que está sendo utilizada neste projeto, o _R²_, maior que os demais algoritmos. Sendo assim, o resultado das métricas gerado pelo treinamento do modelo Random Forest, com o mesmo dataset utilizado pelos dois algoritmos detalhados anteriormente (Regressão Linear e Árvore de Decisão), foi: 
* **Erro Absoluto Médio (MAE):** 1.5389725166963635
* **Erro Quadrático Médio (MSE):** 11.448398912332877
* **Raiz quadrada do erro-médio (RMSE):** 3.383548272499282
* **Coeficiente de Determinação (R²):** 0.6859114017884282

A partir dos resultados acima, optamos por aplicar a otimização por hiperparâmetros para observar se este modelo pode melhorar o seu desempenho. 

No caso do modelo Random Forest, o grupo optou por testar diferentes iterações dentro do algoritmo de otimização de hiperparâmetro **Random Search**. Assim, a escolha de testar diferentes valores está fundamentada no fato de que o modelo Random Forest leva um tempo considerável para processar o algoritmo de otimização, dificultando executar de uma só vez um número alto de iterações. Nesse sentido, ao testar diferentes números de iterações mais baixos, é possível comparar qual apresenta melhor desempenho.

Considerando o que foi exposto no parágrafo acima, será implementado o código referente a otimização de hiperparâmetros por **Randoms Search** com 3, 6 e 9 iterações. 

Para realizar a otimização de hiperparâmetros de forma eficaz, é fundamental adquirir um entendimento abrangente dos hiperparâmetros que afetam o desempenho do modelo Random Forest, bem como compreender suas respectivas definições. Nesse sentido, recorremos às definições propostas por Bruno Faboci [3], as quais esclarecem os seguintes hiperparâmetros:

* **n_estimators:** Este parâmetro representa o "número de árvores na floresta". Ele influencia diretamente a quantidade de árvores de decisão que serão combinadas para formar o ensemble.
* **max_depth:** Trata-se da "profundidade máxima das árvores". A profundidade de uma árvore de decisão determina até que ponto ela pode se ramificar, o que impacta a complexidade do modelo.
* **min_samples_split:** Aqui, nos referimos ao "número mínimo de amostras necessárias para dividir um nó interno". Esse hiperparâmetro controla o quão "criteriosa" a árvore deve ser ao decidir dividir um nó interno em subnós.
* **min_samples_leaf:** Por último, mas não menos importante, temos o "número mínimo de amostras necessárias para estar em um nó folha". Esse hiperparâmetro afeta o tamanho mínimo permitido para um nó folha.

Tendo em mente essas definições cruciais, prosseguimos com as seguintes otimizações:

#### Parâmetros e declaração do modelo

In [ ]:
# Grade de hiperparâmetros
param_rf= {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# EVoca o modelo Random Forest
model_rf = RandomForestRegressor(random_state=42)

#### Otimização por Random Search com três iterações 

In [ ]:
# Adiciona e realiza o random search
random_3 = RandomizedSearchCV(
    estimator=model_rf,
    param_distributions=param_rf, 
    n_iter=3,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

# Treina o modelo com os hiperparâmetros
random_3.fit(X_train, y_train)

In [ ]:
# Encontra os melhores hiperparâmetros
print("Melhores hiperparâmetros encontrados:")
print(random_3.best_params_)

In [ ]:
# Faz a previsão com os melhores hiperparâmetros
best_rf = random_3.best_estimator_
y_pred_random_3 = best_rf.predict(X_test)

In [ ]:
# Avalia com a otimização por random search
mse_random_3 = mean_squared_error(y_test, y_pred_random_3)
r2_random_3 = r2_score(y_test, y_pred_random_3)
mae_random_3 = mean_absolute_error(y_test, y_pred_random_3)
rmse_random_3 = sqrt(mse_random_3 )

print(f"Erro Absoluto Médio (MAE): {mae_random_3}")
print(f"Erro Quadrático Médio (MSE): {mse_random_3}")
print(f"Raiz do Erro Quadrático Médio (RMSE): {rmse_random_3}")
print(f"R2 Score: {r2_random_3}")

#### Otimização por Random Search com seis iterações 


In [ ]:
# Adiciona e realizando o random search
random_6 = RandomizedSearchCV(
    estimator=model_rf,
    param_distributions=param_rf, 
    n_iter=6,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

# Treina o modelo com os hiperparâmetros
random_6.fit(X_train, y_train)

In [ ]:
# Encontra os melhores hiperparâmetros
print("Melhores hiperparâmetros encontrados:")
print(random_6.best_params_)

In [ ]:
# Faz a previsão com os melhores hiperparâmetros
best_rf = random_6.best_estimator_
y_pred_random_6 = best_rf.predict(X_test)

In [ ]:
# Avalia com a otimização por random search
mse_random_6 = mean_squared_error(y_test, y_pred_random_6)
r2_random_6 = r2_score(y_test, y_pred_random_6)
mae_random_6 = mean_absolute_error(y_test, y_pred_random_6)
rmse_random_6 = sqrt(mse_random_6 )

print(f"Erro Absoluto Médio (MAE): {mae_random_6}")
print(f"Erro Quadrático Médio (MSE): {mse_random_6}")
print(f"Raiz do Erro Quadrático Médio (RMSE): {rmse_random_6}")
print(f"R2 Score: {r2_random_6}")

#### Otimização por Random Search com nove iterações 

In [ ]:
# Adiciona e realiza o random search
random_9 = RandomizedSearchCV(
    estimator=model_rf,
    param_distributions=param_rf, 
    n_iter=9,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

# Treina o modelo com os hiperparâmetros
random_9.fit(X_train, y_train)

In [ ]:
# Encontra os melhores hiperparâmetros
print("Melhores hiperparâmetros encontrados:")
print(random_9.best_params_)

In [ ]:
# Faz a previsão com os melhores hiperparâmetros
best_rf = random_9.best_estimator_
y_pred_random_9 = best_rf.predict(X_test)

In [ ]:
# Avalia com a otimização por random search
mse_random_9 = mean_squared_error(y_test, y_pred_random_9)
r2_random_9 = r2_score(y_test, y_pred_random_9)
mae_random_9 = mean_absolute_error(y_test, y_pred_random_9)
rmse_random_9 = sqrt(mse_random_9 )

print(f"Erro Absoluto Médio (MAE): {mae_random_9}")
print(f"Erro Quadrático Médio (MSE): {mse_random_9}")
print(f"Raiz do Erro Quadrático Médio (RMSE): {rmse_random_9}")
print(f"R2 Score: {r2_random_9}")

#### Otimização por Grid Search

Em relação ao algoritmo de **Grid Search**, a decisão tomada foi de aplicá-lo de forma reduzida. Isso porque, como o _dataset_ utilizado para o desenvolvimento deste projeto é extenso, o tempo de execução do algoritmo é muito alto, não tornando viável para o projeto executá-lo em sua integridade.Assim sendo, o `param_grid` (uma grade de parâmetros) foi diminuído, a fim de conseguir executar o **Grid Search** e comparar os seus resultados com os apresentados no algoritmo anterior. O `param_grid` é um conjunto predefinido de combinações de hiperparâmetros que o **Grid Search** explora sistematicamente para encontrar a configuração mais adequada para um modelo de aprendizado de máquina [2]. Reduzir o tamanho do `param_grid` significa considerar menos combinações de hiperparâmetros durante a busca, o que pode economizar tempo computacional.

_[4] Lewis, Christopher. Optimize Hyperparameters with GridSearch. Medium, 7 mai. 2021. Disponível em: https://medium.com/analytics-vidhya/optimize-hyperparameters-with-gridsearch-d351b0fd339d. Acesso em: 21 set. 2023._

In [ ]:
# Os parâmetros de Grid Search a seguir representam aqueles que idealmente testaríamos, baseados naqueles que foram 
# utilizados para o algoritmo de random search.

# from sklearn.model_selection import train_test_split, GridSearchCV
# param_grid = {
#     'n_estimators': [50, 100, 200],      # Número de árvores na floresta
#     'max_depth': [None, 10, 20, 30],    # Profundidade máxima das árvores
#     'min_samples_split': [2, 5, 10],    # Número mínimo de amostras necessárias para dividir um nó
#     'min_samples_leaf': [1, 2, 4]       # Número mínimo de amostras necessárias em uma folha
# }


# Entretanto, como dentro dessas configurações o algoritmo tem um tempo de processamento maior que 10 horas, 
# o grupo optou por reduzir esses parâmetros a fim de apresentar a otimização com Grid Search, resultando no 
# seguinte códgio:

# Grade de hiperparâmetros
from sklearn.model_selection import train_test_split, GridSearchCV
param_grid = {
     'n_estimators': [200],      
     'max_depth': [5],    
     'min_samples_split': [5],    
     'min_samples_leaf': [4]       
}

In [ ]:
# Adiciona e realiza o grid search
grid_search = GridSearchCV(
    estimator=model_rf,
    param_grid=param_grid,
    cv=5, 
    scoring='neg_mean_squared_error', 
    n_jobs=-1,
)

In [ ]:
## Treina o o modelo com os hiperparâmetros
grid_search.fit(X_train, y_train)

In [ ]:
# Encontra os melhores hiperparâmetros
print("Melhores hiperparâmetros encontrados:")
print(grid_search.best_params_)

In [ ]:
# Faz a previsão com os melhores hiperparâmetros
best_rf = grid_search.best_estimator_
y_pred_grid = best_rf.predict(X_test)

In [ ]:
# Avalia com a otimização por grid search
mse_grid= mean_squared_error(y_test, y_pred_grid)
r2_grid = r2_score(y_test, y_pred_grid)
mae_grid = mean_absolute_error(y_test, y_pred_grid)
rmse_grid= sqrt(mse_grid )

print(f"Erro Absoluto Médio (MAE): {mae_grid}")
print(f"Erro Quadrático Médio (MSE): {mse_grid}")
print(f"Raiz do Erro Quadrático Médio (RMSE): {rmse_grid}")
print(f"R2 Score: {r2_grid}")

#### Conclusão

Após aplicarmos o **Random Search** no modelo Random Forest as métricas obtidas por meio da otimização de hiperparâmetros foram:
* **Random Search aplicado ao Random Forest com três iterações:**
  * **Erro Absoluto Médio (MAE):** 1.4559071270701858
  * **Erro Quadrático Médio (MSE):** 8.291097825027418
  * R-squared: 0.7295518366268021
  * **Raiz quadrada do erro-médio (RMSE):** 2.879426648662441
  * **Coeficiente de Determinação (R²):** 0.7295518366268021

&nbsp;

* **Random Search aplicado ao Random Forest com seis iterações:**
  * **Erro Absoluto Médio (MAE):** 1.4544637332616124
  * **Erro Quadrático Médio (MSE):** 8.265656812026746
  * **Raiz quadrada do erro-médio (RMSE):** 2.8750055325210675
  * **Coeficiente de Determinação (R²):** 0.7303816996178786

&nbsp;

* **Random Search aplicado ao Random Forest com nove iterações:**
  * **Erro Absoluto Médio (MAE):** 1.4643446317746063
  * **Erro Quadrático Médio (MSE):** 8.291285890774173
  * **Raiz quadrada do erro-médio (RMSE):** 2.879459305281839
  * **Coeficiente de Determinação (R²):**  0.7295457020910776

É possível observar que o algoritmo de otimização de hiperparâmetro **Random Search** realmente consegue melhorar o desempenho do modelo **Random Fores**, haja vista que a principal métrica escolhida para ser observada, o _R²_, possui, originalmente, o valor de, aproximadamente, 68%. Mas, após ser aplicada a otimização de hiperparâmetro com o **Random Search de seis iterações**, o _R²_ alcançou um valor superior de, aproximadamente, 73%. Isso significa que, de fato, tal algoritmo de otimização de hiperparâmetro consegue otimizar o modelo **Random Forest**. 

Ademais, é válido destacar que os melhores hiperparâmetros encontrados com o **Random Search de seis iterações** foram:
`{'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 20}`

A partir da definição realizada anteriormente, o hiperparâmetro `n_estimators` gerado indica que a quantidade de árvores necessárias para o algoritmo é 200. Já o `min_samples_split` diz que o número mínimo de amostras necessárias para dividir um nó interno é cinco. O `min_samples_leaf` aponta que o número mínimo de amostras necessárias para estar em um nó folha é um e, por fim, o valor de `max_depth` demonstra que a profundidade das árvores é 20.   

As métricas obtidas através do **Grid Search**, aplicado ao modelo de Random Forest, foram:
* **Erro Absoluto Médio (MAE):** 17.80755335081607
* **Erro Quadrático Médio (MSE):** 8.291285890774173
* **Raiz quadrada do erro-médio (RMSE):** 4.219899684923337
* **Coeficiente de Determinação (R²):** 0.4191336057619707

Tendo em vista os resultados das métricas gerados pelo algoritmo de otimização de hiperparâmetros **Grid Search**, é possível observar que não há uma melhora das métricas do algoritmo **Random Forest**, haja vista que a métrica principal, o _R²_, possui o valor de 68% no modelo sem a otimização de hiperparâmetro e o _R²_ gerado pela otimização em questão assume o valor de, aproximadamente, 42%. Diante disso, é notório que o **Grid Search** não otimizou o modelo, mas, sim, piorou os seus resultados. 

Ademais, é válido destacar que os melhores hiperparâmetros encontrados com o **Grid Search** foram:
`{'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 200}`

Com isso, tendo em vista a definição realizada anteriormente, o `max_depth` demonstra que a profundidade das árvores devem ser cinco. Já o `min_samples_leaf` aponta que o número mínimo de amostras necessárias para estar em um nó folha é quatro. O `min_samples_split` diz que o número mínimo de amostras necessárias para dividir um nó interno é cinco e, por fim, o `n_estimators` gerado indica que a quantidade de árvores necessárias para o algoritmo é 200.

## Modelo escolhido e justificativa

Considerando o desempenho dos diferentes modelos, nomeadamente o **Random Forest**, a **Regressão Linear** e a **Árvore de Decisão**, juntamente com as respetivas otimizações de hiperparâmetros, a escolha recaiu sobre o **Random Forest** com otimização através do método de **Random Search**, com um total de 6 iterações. Esta seleção baseia-se na comparação das métricas obtidas anteriormente, revelando-se o mais eficiente, como destacado a seguir:


- **Random Search aplicado ao Random Forest com seis iterações:**
  - **Erro Absoluto Médio (MAE):** 1.4544637332616124
  - **Erro Quadrático Médio (MSE):** 8.265656812026746
  - **Raiz Quadrada do Erro Médio (RMSE):** 2.8750055325210675
  - **Coeficiente de Determinação (R²):** 0.7303816996178786

Os resultados obtidos estão em harmonia com os objetivos específicos do modelo, como explicado em maior detalhe no tópico **4.3.3**. O algoritmo Random Forest é robusto e abrangente, permitindo uma avaliação minuciosa das características do conjunto de dados e fornecendo previsões precisas.

Para uma melhor compreensão da qualidade das previsões efetuadas por este modelo, é necessário uma análise mais aprofundada das métricas. O MAE de cerca de 1.45 e o RMSE de aproximadamente 2.87 indicam que, em média, o modelo pode errar em cerca de 1.45 e 2.87 unidades nas previsões de vendas de produtos, o que é considerado satisfatório, dadas as metas do projeto. O valor de R², que se aproxima de 73%, significa que é possível explicar uma parte significativa da variabilidade das características nos valores alvo, o que também é relevante para os objetivos do projeto. O RMSE, que penaliza erros mais significativos, situa-se em cerca de 8.26, indicando que existem previsões que se desviam consideravelmente dos valores reais, sendo um ponto a ter em consideração, embora não represente uma queda drástica no desempenho.

Portanto, com base na análise anteriormente realizada, o modelo selecionado efetua previsões satisfatórias do número de vendas da empresa Mobly, sendo essencial para o propósito de otimizar a gestão de estoque e melhorar a eficiência das operações de entrega.